# Real-World Crime Analytics for the London Metropolitan Area

## 1. Environment Setup (Linux)

### 1.1. Update Linux Packages

In [ ]:
!sudo apt update

### 1.2. Install Python and Packages (pip & venv)

In [ ]:
!sudo apt install -y python3 python3-pip python3-venv

### 1.3. Create a Virtual Environment

In [ ]:
!python3 -m venv venv

### 1.4. Activate the Virtual Environment

In [ ]:
!source venv/bin/activate

### 1.5. Install the Required Python Packages

In [ ]:
%pip install -r requirements.txt

## 2. Data Ingestion

### 2.1. Create Folders to Store Data

In [ ]:
import os

# Create datasets folder if it doesn't exist
os.makedirs("Datasets", exist_ok=True)
os.makedirs("Datasets/Raw-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Map-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Police-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Income-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Population-Data", exist_ok=True)

### 2.2. Download Data

#### 2.2.1. Download Income Data - <a>www.ons.gov.uk</a>

In [ ]:
!wget -O Datasets/Raw-Data/Income-Data/Income-MSOA.xlsx "https://www.ons.gov.uk/file?uri=/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/smallareaincomeestimatesformiddlelayersuperoutputareasenglandandwales/financialyearending2020/saiefy1920finalqaddownload280923.xlsx"
print("Downloaded to Datasets/Raw-Data/Income-Data/Income-MSOA.xlsx")

#### 2.2.2. Download Population Data - <a>www.ons.gov.uk</a>

In [ ]:
!wget -O Datasets/Raw-Data/Population-Data/Population-LSOA.xlsx "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/lowersuperoutputareamidyearpopulationestimates/mid2022revisednov2025tomid2024/sapelsoasyoa20222024.xlsx"
print("Downloaded to Datasets/Raw-Data/Population-Data/Population-LSOA.xlsx")

#### 2.2.3. Download Police Data - <a>data.police.uk</a>

In [ ]:
!wget -O Datasets/Raw-Data/Police-Data/RAW-POLICE-2022-10-TO-2025-09.zip "https://data.police.uk/data/archive/2025-09.zip"
print("Downloaded to Datasets/Raw-Data/RAW-POLICE-2022-10-TO-2025-09.zip")

In [ ]:
# Extract the police data zip file
!unzip -o Datasets/Raw-Data/RAW-POLICE-2022-10-TO-2025-09.zip -d Datasets/Raw-Data/Police-Data
print("Extracted police data to Datasets/Raw-Data/Police-Data/")

In [ ]:
# Clean up the zip file to save space
!rm -rf Datasets/Raw-Data/RAW-POLICE-2022-10-TO-2025-09.zip

#### 2.2.4. Download Map Data - <a>geoportal.statistics.gov.uk</a>

#### Note: Map Data from ONS Geography Can't be Downloaded Directly, So Use the Below Links to Download Them.

1. LSOAs - <a>https://geoportal.statistics.gov.uk/datasets/ons::output-areas-december-2021-boundaries-ew-bgc-v2/about</a>

2. MSOAs - <a>https://geoportal.statistics.gov.uk/datasets/ons::middle-layer-super-output-areas-december-2021-boundaries-ew-bfc-v7-2/about</a>

### 2.3. Converting File Types and Keeping Necessary Files

#### 2.3.1. Convert XLSX (Excel) files to CSV using pandas

In [ ]:
import pandas as pd
import os
import re

# Safe filename from sheet name
def sanitize(name: str) -> str:
    # Replace non-alphanumeric with underscores, strip, collapse repeats
    name = re.sub(r"[^A-Za-z0-9]+", "_", name).strip("_")
    return re.sub(r"_+", "_", name) or "Sheet"

# Convert Income-MSOA.xlsx: write one CSV per sheet
income_xlsx_path = "Datasets/Raw-Data/Income-Data/Income-MSOA.xlsx"
income_out_dir = "Datasets/Raw-Data/Income-Data/Income-MSOA"
os.makedirs(income_out_dir, exist_ok=True)

try:
    xls_income = pd.ExcelFile(income_xlsx_path)
    for sheet in xls_income.sheet_names:
        df = pd.read_excel(xls_income, sheet_name=sheet)
        safe = sanitize(sheet)
        out_path = os.path.join(income_out_dir, f"Income-MSOA-{safe}.csv")
        df.to_csv(out_path, index=False)
    print(f"Exported {len(xls_income.sheet_names)} sheets from {income_xlsx_path} to {income_out_dir}")
except Exception as e:
    print("Failed to process income workbook:", e)

# Convert Population-LSOA.xlsx: write one CSV per sheet
population_xlsx_path = "Datasets/Raw-Data/Population-Data/Population-LSOA.xlsx"
population_out_dir = "Datasets/Raw-Data/Population-Data/Population-LSOA"
os.makedirs(population_out_dir, exist_ok=True)

try:
    xls_pop = pd.ExcelFile(population_xlsx_path)
    for sheet in xls_pop.sheet_names:
        df = pd.read_excel(xls_pop, sheet_name=sheet)
        safe = sanitize(sheet)
        out_path = os.path.join(population_out_dir, f"Population-LSOA-{safe}.csv")
        df.to_csv(out_path, index=False)
    print(f"Exported {len(xls_pop.sheet_names)} sheets from {population_xlsx_path} to {population_out_dir}")
except Exception as e:
    print("Failed to process population workbook:", e)

#### 2.3.2. Retain Only Metropolitan Police Records

In [ ]:
from pathlib import Path
from typing import List

police_root = Path("Datasets/Raw-Data/Police-Data")
date_range = ("2022-10", "2025-09")
keep_token = "metropolitan"

if not police_root.exists():
    raise FileNotFoundError(f"Missing directory: {police_root}")

kept: List[Path] = []
removed: List[Path] = []

# Delete every police file that does not belong to the Metropolitan force.
for file_path in police_root.rglob("*"):
    if not file_path.is_file():
        continue
    if keep_token in file_path.name.lower():
        kept.append(file_path)
        continue
    file_path.unlink()
    removed.append(file_path)

print(f"Date Range - {date_range[0]} - {date_range[1]}")
for path in sorted(kept):
    print(path)

print(f"Removed {len(removed)} other files.")

### 2.4. Copy Only the Necessary Files for Cleaning

In [ ]:
os.makedirs("Datasets/Data-for-Cleaning", exist_ok=True)
os.makedirs("Datasets/Data-for-Cleaning/Police-Data", exist_ok=True)
os.makedirs("Datasets/Data-for-Cleaning/Income-Data", exist_ok=True)
os.makedirs("Datasets/Data-for-Cleaning/Population-Data", exist_ok=True)
os.makedirs("Datasets/Data-for-Cleaning/Map-Data", exist_ok=True)

#### 2.4.1. Copy Police Data

In [ ]:
!cp -r Datasets/Raw-Data/Police-Data/* Datasets/Data-for-Cleaning/Police-Data/

#### 2.4.2. Copy Map Data

In [ ]:
!cp -r Datasets/Raw-Data/Map-Data/* Datasets/Data-for-Cleaning/Map-Data/

#### 2.4.3. Copy Income Data

In [ ]:
!cp -r Datasets/Raw-Data/Income-Data/Income-MSOA/Income-MSOA-Total_annual_income.csv Datasets/Data-for-Cleaning/Income-Data/

#### 2.4.4. Copy Population Data

In [ ]:
!cp -r Datasets/Raw-Data/Population-Data/Population-LSOA/Population-LSOA-Mid_2022_LSOA_2021.csv Datasets/Data-for-Cleaning/Population-Data/
!cp -r Datasets/Raw-Data/Population-Data/Population-LSOA/Population-LSOA-Mid_2023_LSOA_2021.csv Datasets/Data-for-Cleaning/Population-Data/
!cp -r Datasets/Raw-Data/Population-Data/Population-LSOA/Population-LSOA-Mid_2024_LSOA_2021.csv Datasets/Data-for-Cleaning/Population-Data/

## 3. Data Cleaning

### 3. 1. Initialize Spark

In [ ]:
# Import and initialize Spark
import pyspark
from pathlib import Path
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import NumericType


# Initialize session
spark = SparkSession.builder \
    .appName("Real-World Crime Analytics for the London Metropolitan Area") \
    .master("local[*]") \
    .getOrCreate()

print("Spark version:", spark.version)

### 3. 2. Cleaning Income Data

#### 3.2.1.  Reading the Income Data - Data & Schema

In [ ]:
# Load and display the total annual income data
total_income_path = "Datasets/Data-for-Cleaning/Income-Data/Income-MSOA-Total_annual_income.csv"

total_income_df = spark.read.option("header", "true").csv(total_income_path)

total_income_df.show(5)
total_income_df.printSchema()

#### 3.2.2.  Remove Extra Header Lines - Income Data

In [ ]:
# read as text so we can inspect the actual first lines
raw = spark.read.text(total_income_path)

# Drop the first 4 lines
clean_lines = raw.rdd.zipWithIndex() \
    .filter(lambda x: x[1] >= 4) \
    .map(lambda x: x[0].value)

# Convert back to CSV DataFrame
clean_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(clean_lines)

total_income_df = clean_df

total_income_df.show(5)
total_income_df.printSchema()

#### 3.2.3. Summary Statistics for Cleaned Income Data

In [ ]:
total_income_df.describe().show()

#### 3.2.4. Detecting Duplicate Rows in Cleaned Income Data


In [ ]:
duplicates = total_income_df.groupBy(total_income_df.columns) \
               .count() \
               .filter("count > 1")

duplicates.show()

### 3. 3. Cleaning Population Data

#### 3.3.1.  Reading the Population Data - Data & Schema

In [ ]:
population_paths = {
    "2022": "Datasets/Data-for-Cleaning/Population-Data/Population-LSOA-Mid_2022_LSOA_2021.csv",
    "2023": "Datasets/Data-for-Cleaning/Population-Data/Population-LSOA-Mid_2023_LSOA_2021.csv",
    "2024": "Datasets/Data-for-Cleaning/Population-Data/Population-LSOA-Mid_2024_LSOA_2021.csv",
}

population_dfs = {}
for year, path in population_paths.items():
    df = spark.read.option("header", "true").csv(path)
    population_dfs[year] = df
    print(f"Population {year} Data:")
    df.show(5)
    df.printSchema()

population_2022_df = population_dfs["2022"]
population_2023_df = population_dfs["2023"]
population_2024_df = population_dfs["2024"]

#### 3.3.2.  Remove Extra Header Lines - Population Data

In [ ]:
def clean_population(path: str):
    rows = spark.read.text(path)
    trimmed = rows.rdd.zipWithIndex().filter(lambda x: x[1] >= 3).map(lambda x: x[0].value)
    return spark.read.option("header", "true").option("inferSchema", "true").csv(trimmed)

population_clean = {year: clean_population(path) for year, path in population_paths.items()}

population_2022_df = population_clean["2022"]
population_2023_df = population_clean["2023"]
population_2024_df = population_clean["2024"]

for year, df in population_clean.items():
    print(f"Cleaned Population {year} Data:")
    df.show(5)
    df.printSchema()

#### 3.3.3. Summary Statistics for Cleaned Population Data

In [ ]:
print("Population 2022 Data:")
population_2022_df.describe().show()

print("Population 2023 Data:")
population_2023_df.describe().show()

print("Population 2024 Data:")
population_2024_df.describe().show()

#### 3.3.4. Detecting Duplicate Rows in Cleaned Population Data


In [ ]:
duplicates = {
    year: df.groupBy(*df.columns).count().filter("count > 1")
    for year, df in population_clean.items()
}

duplicates_2022 = duplicates["2022"]
duplicates_2023 = duplicates["2023"]
duplicates_2024 = duplicates["2024"]

for year, dup_df in duplicates.items():
    print(f"Duplicate Rows in Population {year} Data:")
    dup_df.show()

### 3. 4. Cleaning Map Data

#### 3.4.1.  Reading the Map Data - Data & Schema

In [ ]:
# Load and display the population datasets
map_paths = {
    "lsoa": "Datasets/Data-for-Cleaning/Map-Data/Map-LSOA-2021.csv",
    "msoa": "Datasets/Data-for-Cleaning/Map-Data/Map-MSOA-2021.csv",
}

map_dfs = {}
for map, path in map_paths.items():
    df = spark.read.option("header", "true").csv(path)
    map_dfs[map] = df
    print(f"Population {map} Data:")
    df.show(5)
    df.printSchema()

map_lsoa_df = map_dfs["lsoa"]
map_msoa_df = map_dfs["msoa"]


#### 3.4.2. Summary Statistics for Map Data

In [ ]:
print("Map LSOA Data:")
map_lsoa_df.describe().show()

print("Map MSOA Data:")
map_msoa_df.describe().show()


#### 3.4.3. Detecting Duplicate Rows in Map Data


In [ ]:
duplicates_map = {
    "lsoa": map_lsoa_df.groupBy(*map_lsoa_df.columns).count().filter("count > 1"),
    "msoa": map_msoa_df.groupBy(*map_msoa_df.columns).count().filter("count > 1"),
}

duplicates_lsoa = duplicates_map["lsoa"]
duplicates_msoa = duplicates_map["msoa"]

for map_type, dup_df in duplicates_map.items():
    print(f"Duplicate Rows in Map {map_type.upper()} Data:")
    dup_df.show()

### 3. 5. Cleaning Police Data

#### 3.5.1. Combine All Police data into a Single Frame


In [ ]:

police_data_root = Path("Datasets/Data-for-Cleaning/Police-Data")

def load_police_dataset(file_glob: str) -> DataFrame:
    """Load all monthly police CSVs matching the glob into a single DataFrame."""
    # Gather every CSV matching the pattern across the month folders.
    matches = sorted(police_data_root.rglob(file_glob))
    if not matches:
        raise FileNotFoundError(f"No police files matched pattern: {file_glob}")
    print(f"Matched {len(matches)} files for pattern '{file_glob}'")
    # Load all matched files into a single Spark DataFrame.
    df = spark.read.option("header", "true").option("inferSchema", "true").csv([str(path) for path in matches])

    return df

# Build combined DataFrames for each police dataset family.
police_outcomes_df = load_police_dataset("*-metropolitan-outcomes.csv")
police_stop_search_df = load_police_dataset("*-metropolitan-stop-and-search.csv")
police_street_df = load_police_dataset("*-metropolitan-street.csv")

# Register temp views for downstream Spark SQL operations.
police_outcomes_df.createOrReplaceTempView("police_outcomes")
police_stop_search_df.createOrReplaceTempView("police_stop_and_search")
police_street_df.createOrReplaceTempView("police_street")

# Print simple row counts for a quick sanity check.
print("Combined police outcomes rows:", police_outcomes_df.count())
print("Combined police stop and search rows:", police_stop_search_df.count())
print("Combined police street rows:", police_street_df.count())

#### 3.5.2. Police Outcomes Data Check

##### 3.5.2.1. Police Outcomes Schema Check

In [ ]:
print("Police Outcomes Schema:")
police_outcomes_df.printSchema()

##### 3.5.2.2. Police Outcomes Data Rows Check

In [ ]:
print("First 5 Records")
police_outcomes_df.orderBy("Month").show(5)

print("Last 5 Records")
police_outcomes_df.orderBy("Month").tail(5)

##### 3.5.2.3. Police Outcomes Values Check

In [ ]:
police_outcomes_df.describe().show()

##### 3.5.2.4. Fill missing data with readable placeholders.


In [ ]:
# Fill all missing values in police_outcomes_df with None
police_outcomes_df = police_outcomes_df.fillna("None")

# Detect numeric columns automatically
numeric_cols_outcomes = [
    f.name for f in police_outcomes_df.schema.fields
    if isinstance(f.dataType, NumericType)
]

# Fill all numeric columns with 0.0
if numeric_cols_outcomes:
    police_outcomes_df = police_outcomes_df.fillna(0.0, subset=numeric_cols_outcomes)

police_outcomes_df.describe().show()

#### 3.5.3. Police Stop and Search Data Check

##### 3.5.3.1. Police Stop and Search Schema Check

In [ ]:
print("Police Stop and Search Schema:")
police_stop_search_df.printSchema()

##### 3.5.3.2. Police Stop and Search Data Rows Check

In [ ]:
print("First 5 Records")
police_stop_search_df.orderBy("Date").show(5)

print("Last 5 Records")
police_stop_search_df.orderBy("Date").tail(5)

##### 3.5.3.3. Police Stop and Search Values Check

In [ ]:
police_stop_search_df.describe().show()

##### 3.5.3.4. Fill missing data with readable placeholders.


In [ ]:
# Fill all missing values in police_stop_search_df with None
police_stop_search_df = police_stop_search_df.fillna("None")

# Detect numeric columns automatically
numeric_cols_stop = [
    f.name for f in police_stop_search_df.schema.fields
    if isinstance(f.dataType, NumericType)
]

# Fill all numeric columns with 0.0
if numeric_cols_stop:
    police_stop_search_df = police_stop_search_df.fillna(0.0, subset=numeric_cols_stop)

police_stop_search_df.describe().show()

#### 3.5.4. Police Street Data Check

##### 3.5.4.1. Police Street Schema Check

In [ ]:
print("Police Stop and Search Schema:")
police_street_df.printSchema()

##### 3.5.4.2. Police Street Data Rows Check

In [ ]:
print("First 5 Records")
police_street_df.orderBy("Month").show(5)

print("Last 5 Records")
police_street_df.orderBy("Month").tail(5)

##### 3.5.4.3. Police Street Values Check

In [ ]:
police_street_df.describe().show()

##### 3.5.4.4. Fill missing data with readable placeholders.


In [ ]:
# Fill all missing values in police_street_df with None
police_street_df = police_street_df.fillna("None")

# Detect numeric columns automatically
numeric_cols_street = [
    f.name for f in police_street_df.schema.fields
    if isinstance(f.dataType, NumericType)
]

# Fill all numeric columns with 0.0
if numeric_cols_street:
    police_street_df = police_street_df.fillna(0.0, subset=numeric_cols_street)

police_street_df.describe().show()

## 4. Data Transformation

### 4.0. Create Folders for Cleaned Data

In [ ]:
import os

os.makedirs("Datasets/Cleaned-Data/Police-Data", exist_ok=True)
os.makedirs("Datasets/Cleaned-Data/Income-Data", exist_ok=True)
os.makedirs("Datasets/Cleaned-Data/Population-Data", exist_ok=True)
os.makedirs("Datasets/Cleaned-Data/Map-Data", exist_ok=True)

### 4.1. Transform Cleaned Income Dataset

In [ ]:
total_income_df.show(5)

#### 4.1.1. Select only the required columns from clean_df


In [ ]:
selected_income_df = total_income_df.select(
    "MSOA code",
    "MSOA name",
    "Total annual income (£)"
)

selected_income_df.show(5)

#### 4.1.2. Standardize Colomn Names 

In [ ]:
def simplify_column_names(df):
    new_cols = [col.lower().replace(" ", "_") for col in df.columns]
    return df.toDF(*new_cols)

selected_income_df = simplify_column_names(selected_income_df)
selected_income_df.show(5)

#### 4.1.3. Verify Income Data Schema

In [ ]:
selected_income_df.printSchema()

#### 4.1.3. Save the cleaned income data

In [ ]:
selected_income_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Income-Data/")

In [ ]:
total_income_df = selected_income_df

### 4.2. Transform Cleaned Population Dataset

#### 4.2.1. Display the Population DataFrames

In [ ]:
population_2022_df.show(5)
population_2023_df.show(5)
population_2024_df.show(5)

#### 4.2.2. Select Required Columns from Population Data

In [ ]:
def select_population_columns(population_df):
    # Select only the required columns from the population DataFrame
    return population_df.select(
        "LSOA 2021 Code",
        "LSOA 2021 Name",
        "Total"

    )

selected_population_2022_df = select_population_columns(population_2022_df)
selected_population_2023_df = select_population_columns(population_2023_df)
selected_population_2024_df = select_population_columns(population_2024_df)

selected_population_2022_df.show(5)
selected_population_2023_df.show(5)
selected_population_2024_df.show(5)


#### 4.2.3. Standardize Population Column Names

In [ ]:
def simplify_column_names(df):
    new_cols = [col.lower().replace(" ", "_") for col in df.columns]
    return df.toDF(*new_cols)

selected_population_2022_df = simplify_column_names(selected_population_2022_df)
selected_population_2022_df.show(5)

selected_population_2023_df = simplify_column_names(selected_population_2023_df)
selected_population_2023_df.show(5)

selected_population_2024_df = simplify_column_names(selected_population_2024_df)
selected_population_2024_df.show(5)

#### 4.2.4. Verify Population Data Schema

In [ ]:
selected_population_2022_df.printSchema()
selected_population_2023_df.printSchema()
selected_population_2024_df.printSchema()

In [ ]:
from pyspark.sql.functions import lit

# Add year column to each population dataframe
pop_2022_with_year = selected_population_2022_df.withColumn("year", lit(2022))
pop_2023_with_year = selected_population_2023_df.withColumn("year", lit(2023))
pop_2024_with_year = selected_population_2024_df.withColumn("year", lit(2024))

# Union all population dataframes
combined_population_df = pop_2022_with_year.union(pop_2023_with_year).union(pop_2024_with_year)

combined_population_df.show(5)

#### 4.2.5. Save Cleaned Population Data

In [ ]:
# Save the cleaned income data
selected_population_2022_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Population-Data/2022")
selected_population_2023_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Population-Data/2023")
selected_population_2024_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Population-Data/2024")
combined_population_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Population-Data/Combined")                                                                                        

In [ ]:
population_2022_df = selected_population_2022_df
population_2023_df = selected_population_2023_df
population_2024_df = selected_population_2024_df
population_df = combined_population_df

### 4.3. Transform Cleaned Map Dataset

#### 4.3.1. Display the Map DataFrames

In [ ]:
map_lsoa_df.show(5)
map_msoa_df.show(5)

#### 4.3.2. Select Required Columns from LSOA Map Data

In [ ]:
def select_lsoa_columns(map_lsoa_df):
    # Select only the required columns from the population DataFrame
    return map_lsoa_df.select(
        "LSOA21CD",
        "LSOA21NM",
        "LAT",
        "LONG"
    )

selected_map_lsoa_df = select_lsoa_columns(map_lsoa_df)

selected_map_lsoa_df.show(5)

#### 4.3.3. Select Required Columns from MSOA Map Data

In [ ]:
def select_msoa_columns(map_msoa_df):
    # Select only the required columns from the population DataFrame
    return map_msoa_df.select(
        "MSOA21CD",
        "MSOA21NM",
        "LAT",
        "LONG"
    )

selected_map_msoa_df = select_msoa_columns(map_msoa_df)

selected_map_msoa_df.show(5)


#### 4.3.4. Standardize Map Column Names

In [ ]:
def simplify_column_names(df):
    new_cols = [col.lower().replace(" ", "_") for col in df.columns]
    return df.toDF(*new_cols)

selected_map_lsoa_df = simplify_column_names(selected_map_lsoa_df)
selected_map_lsoa_df.show(5)

selected_map_msoa_df = simplify_column_names(selected_map_msoa_df)
selected_map_msoa_df.show(5)


#### 4.3.5. Verify Map Data Schema

In [ ]:
selected_map_lsoa_df.printSchema()
selected_map_msoa_df.printSchema()

#### 4.3.6. Save Cleaned Map Data

In [ ]:
# Save the cleaned income data
selected_map_lsoa_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Map-Data/lsoa")
selected_map_msoa_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Map-Data/msoa")

In [ ]:
map_msoa_df = selected_map_msoa_df
map_lsoa_df = selected_map_lsoa_df

### 4.4. Transform Cleaned Police Dataset

#### 4.4.1. Display the Police DataFrames

In [ ]:
police_outcomes_df.show(5)
police_stop_search_df.show(5)
police_street_df.show(5)

#### 4.4.2. Standardize Police Data Column Names

In [ ]:
def simplify_column_names(df):
    new_cols = [col.lower().replace(" ", "_") for col in df.columns]
    return df.toDF(*new_cols)

police_outcomes_df = simplify_column_names(police_outcomes_df)
police_outcomes_df.show(5)

police_stop_search_df = simplify_column_names(police_stop_search_df)
police_stop_search_df.show(5)

police_street_df = simplify_column_names(police_street_df)
police_street_df.show(5)

#### 4.4.3. Save Police Data as CSV 

In [ ]:
# Save the cleaned income data
# police_stop_search_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Police-Data/stop-search")
# police_outcomes_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Police-Data/outcomes")
# police_street_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Police-Data/street")

#### 4.4.4. Save Police Data as Parquet with Snappy Compression 

In [ ]:
# Save stop & search data as Parquet
police_stop_search_df.write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("Datasets/Cleaned-Data/Police-Data/stop-search-parquet")

# Save outcomes data as Parquet
police_outcomes_df.write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("Datasets/Cleaned-Data/Police-Data/outcomes-parquet")

# Save street data as Parquet
police_street_df.write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("Datasets/Cleaned-Data/Police-Data/street-parquet")


## 5. Data Querying

In [ ]:
from pyspark.sql.functions import year, col, count
from pyspark.sql.functions import lit, avg, sum as spark_sum
from pyspark.sql.window import Window

### 5.1. Street Crime Analysis

#### 5.1.1. Total Street Crimes by Year

In [104]:
police_street_df.createOrReplaceTempView("police_street")

crimes_by_year = spark.sql("""
SELECT
  YEAR(TO_DATE(month, 'yyyy-MM')) AS year,
  COUNT(*) AS total_crimes
FROM police_street
WHERE month IS NOT NULL
GROUP BY YEAR(TO_DATE(month, 'yyyy-MM'))
ORDER BY year
""")

print("Total Crimes by Year:")
crimes_by_year.show()

Total Crimes by Year:


+----+------------+
|year|total_crimes|
+----+------------+
|2022|      284383|
|2023|     1135031|
|2024|     1139329|
|2025|      856701|
+----+------------+



#### 5.1.2. Total Street Crimes by Year with Cumulative Totals and Percentage

In [105]:
crimes_by_year.createOrReplaceTempView("crimes_by_year")

crimes_by_year_with_analysis = spark.sql("""
SELECT
  year,
  total_crimes,
  SUM(total_crimes) OVER (
    ORDER BY year
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS cumulative_crimes,
  ROUND(
    (total_crimes / SUM(total_crimes) OVER ()) * 100,
    2
  ) AS percentage_of_total
FROM crimes_by_year
ORDER BY year
""")

print("\nTotal Crimes by Year with Analysis:")
crimes_by_year_with_analysis.show()


Total Crimes by Year with Analysis:


25/12/13 08:49:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 0

+----+------------+-----------------+-------------------+
|year|total_crimes|cumulative_crimes|percentage_of_total|
+----+------------+-----------------+-------------------+
|2022|      284383|           284383|               8.33|
|2023|     1135031|          1419414|              33.23|
|2024|     1139329|          2558743|              33.36|
|2025|      856701|          3415444|              25.08|
+----+------------+-----------------+-------------------+



25/12/13 08:49:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 0

#### 5.1.3. Total Outcomes by Year

In [106]:
police_outcomes_df.createOrReplaceTempView("police_outcomes")

outcomes_by_year = spark.sql("""
SELECT
  YEAR(TO_DATE(month, 'yyyy-MM')) AS year,
  COUNT(*) AS total_outcomes
FROM police_outcomes
WHERE month IS NOT NULL
GROUP BY YEAR(TO_DATE(month, 'yyyy-MM'))
ORDER BY year
""")

print("Total Outcomes by Year:")
outcomes_by_year.show()

Total Outcomes by Year:


+----+--------------+
|year|total_outcomes|
+----+--------------+
|2022|         91647|
|2023|        996983|
|2024|       1027756|
|2025|        678038|
+----+--------------+



#### 5.1.4. Total Outcomes by Year with Cumulative Totals and Percentage

In [107]:
outcomes_by_year.createOrReplaceTempView("outcomes_by_year")

outcomes_by_year_with_analysis = spark.sql("""
SELECT
  year,
  total_outcomes,
  SUM(total_outcomes) OVER (
    ORDER BY year
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS cumulative_outcomes,
  ROUND(
    (total_outcomes / SUM(total_outcomes) OVER ()) * 100,
    2
  ) AS percentage_of_total
FROM outcomes_by_year
ORDER BY year
""")

print("\nTotal Outcomes by Year with Analysis:")
outcomes_by_year_with_analysis.show()


Total Outcomes by Year with Analysis:


25/12/13 08:49:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----+--------------+-------------------+-------------------+
|year|total_outcomes|cumulative_outcomes|percentage_of_total|
+----+--------------+-------------------+-------------------+
|2022|         91647|              91647|               3.28|
|2023|        996983|            1088630|              35.68|
|2024|       1027756|            2116386|              36.78|
|2025|        678038|            2794424|              24.26|
+----+--------------+-------------------+-------------------+



25/12/13 08:49:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 08:49:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/13 0

#### 5.1.5. Outcomes vs Street Crimes


In [108]:
print("\nOutcomes vs Crimes Comparison:")

# Ensure both DataFrames are available as SQL views
crimes_by_year.createOrReplaceTempView("crimes_by_year")
outcomes_by_year.createOrReplaceTempView("outcomes_by_year")

comparison_df = spark.sql("""
SELECT
  c.year AS year,
  c.total_crimes,
  o.total_outcomes,
  ROUND(
    CASE
      WHEN c.total_crimes > 0 THEN (o.total_outcomes / c.total_crimes) * 100
      ELSE 0
    END,
    2
  ) AS `outcome_rate_%`
FROM crimes_by_year c
INNER JOIN outcomes_by_year o
  ON c.year = o.year
ORDER BY year
""")

comparison_df.show()


Outcomes vs Crimes Comparison:


+----+------------+--------------+--------------+
|year|total_crimes|total_outcomes|outcome_rate_%|
+----+------------+--------------+--------------+
|2022|      284383|         91647|         32.23|
|2023|     1135031|        996983|         87.84|
|2024|     1139329|       1027756|         90.21|
|2025|      856701|        678038|         79.15|
+----+------------+--------------+--------------+



#### 5.1.6. Avarage Population by Location

In [117]:
population_df.createOrReplaceTempView("population")

avg_population_by_lsoa = spark.sql("""
SELECT
  lsoa_2021_code,
  lsoa_2021_name,
  AVG(try_cast(regexp_replace(cast(total AS string), ',', '') AS double)) AS avg_population
FROM population
GROUP BY lsoa_2021_code, lsoa_2021_name
ORDER BY avg_population DESC
""")

print("\nAverage Population by LSOA:")
avg_population_by_lsoa.show(5, truncate=False)



Average Population by LSOA:


+--------------+------------------+-----------------+
|lsoa_2021_code|lsoa_2021_name    |avg_population   |
+--------------+------------------+-----------------+
|E01028521     |Oxford 008A       |9817.666666666666|
|E01035514     |Cambridge 005H    |8611.333333333334|
|E01033617     |Birmingham 050F   |6959.0           |
|E01013378     |York 023B         |6785.666666666667|
|E01034493     |County Durham 030H|6463.0           |
+--------------+------------------+-----------------+
only showing top 5 rows


#### 5.1.7. Top 5 Street Crimes by Location

In [118]:
police_street_df.createOrReplaceTempView("police_street")

crimes_by_lsoa = spark.sql("""
SELECT
  lsoa_code,
  lsoa_name,
  COUNT(*) AS total_crimes
FROM police_street
GROUP BY lsoa_code, lsoa_name
ORDER BY total_crimes DESC
""")

print("\nTop 5 Locations by Crime Count:")
crimes_by_lsoa.show(5, truncate=False)



Top 5 Locations by Crime Count:


+---------+----------------+------------+
|lsoa_code|lsoa_name       |total_crimes|
+---------+----------------+------------+
|E01035716|Westminster 013G|36888       |
|E01004763|Westminster 013B|33895       |
|E01004734|Westminster 018A|33164       |
|None     |None            |31291       |
|E01004736|Westminster 018C|18857       |
+---------+----------------+------------+
only showing top 5 rows


#### 5.1.8. Top 5 Locations by Street Crime with Population

In [119]:
crimes_by_lsoa.createOrReplaceTempView("crimes_by_lsoa")
avg_population_by_lsoa.createOrReplaceTempView("avg_population_by_lsoa")

crime_population_analysis = spark.sql("""
SELECT
  c.lsoa_code,
  c.lsoa_name,
  c.total_crimes,
  p.avg_population,
  CASE
    WHEN p.avg_population IS NOT NULL AND p.avg_population > 0
      THEN ROUND((c.total_crimes / p.avg_population) * 1000, 2)
    ELSE 0
  END AS crimes_per_1000_people
FROM crimes_by_lsoa c
LEFT JOIN avg_population_by_lsoa p
  ON c.lsoa_code = p.lsoa_2021_code
ORDER BY c.total_crimes DESC
""")

print("\nTop 5 Locations by Crime with Population Analysis:")
crime_population_analysis.show(5, truncate=False)


Top 5 Locations by Crime with Population Analysis:


+---------+----------------+------------+------------------+----------------------+
|lsoa_code|lsoa_name       |total_crimes|avg_population    |crimes_per_1000_people|
+---------+----------------+------------+------------------+----------------------+
|E01035716|Westminster 013G|36888       |2270.0            |16250.22              |
|E01004763|Westminster 013B|33895       |2187.3333333333335|15496.04              |
|E01004734|Westminster 018A|33164       |1806.0            |18363.23              |
|None     |None            |31291       |NULL              |0.0                   |
|E01004736|Westminster 018C|18857       |1609.0            |11719.7               |
+---------+----------------+------------+------------------+----------------------+
only showing top 5 rows


#### 5.1.9. Top 5 by Crime Rate Per Capita

In [121]:
print("\nTop 5 Locations by Crime Rate (per 1000 people):")

crime_population_analysis.createOrReplaceTempView("crime_population_analysis")

spark.sql("""
SELECT
  lsoa_code,
  lsoa_name,
  total_crimes,
  avg_population,
  crimes_per_1000_people
FROM crime_population_analysis
ORDER BY crimes_per_1000_people DESC
LIMIT 5
""").show(truncate=False)


Top 5 Locations by Crime Rate (per 1000 people):


+---------+----------------+------------+------------------+----------------------+
|lsoa_code|lsoa_name       |total_crimes|avg_population    |crimes_per_1000_people|
+---------+----------------+------------+------------------+----------------------+
|E01004734|Westminster 018A|33164       |1806.0            |18363.23              |
|E01035716|Westminster 013G|36888       |2270.0            |16250.22              |
|E01004763|Westminster 013B|33895       |2187.3333333333335|15496.04              |
|E01004736|Westminster 018C|18857       |1609.0            |11719.7               |
|E01004714|Westminster 011B|12948       |1723.0            |7514.8                |
+---------+----------------+------------+------------------+----------------------+



### 5.2. Stop and Search Analysis

#### 5.2.1. Total Stop and Search by Year

In [ ]:
stop_search_by_year = police_stop_search_df \
    .withColumn("year", F.year(F.to_date(col("date"), "yyyy-MM-dd"))) \
    .groupBy("year") \
    .agg(F.count("*").alias("total_stop_searches")) \
    .orderBy("year")

stop_search_by_year.show()

#### 5.2.1. Total Stop and Search by Year with Cumulative Totals and Percentage

In [ ]:
window_spec = Window.orderBy("year").rowsBetween(Window.unboundedPreceding, Window.currentRow)

stop_search_analysis = stop_search_by_year \
    .withColumn("cumulative_searches", F.sum("total_stop_searches").over(window_spec)) \
    .withColumn("percentage_of_total", 
                F.round((col("total_stop_searches") / F.sum("total_stop_searches").over(Window.partitionBy())) * 100, 2))

print("\nStop and Search by Year with Analysis:")
stop_search_analysis.show()


#### 5.2.3. Total searches by Type

In [ ]:
stop_search_by_type_summary = police_stop_search_df \
    .groupBy("type") \
    .agg(F.count("*").alias("total_searches")) \
    .orderBy(F.desc("total_searches"))

print("\nSummary by Type:")
stop_search_by_type_summary.show(truncate=False)

#### 5.2.4. Stop and Search by Type/Object of Search

In [ ]:
stop_search_by_type = police_stop_search_df \
    .groupBy("type", "object_of_search") \
    .agg(F.count("*").alias("total_searches")) \
    .orderBy(F.desc("total_searches"))

print("\nTop 10 Search Types and Objects:")
stop_search_by_type.show(10, truncate=False)

#### 5.2.5. Stop and Searches By Age Range only

In [ ]:
stop_search_by_age = police_stop_search_df \
    .groupBy("age_range") \
    .agg(F.count("*").alias("total_searches")) \
    .orderBy(F.desc("total_searches"))

print("\nSummary by Age Range:")
stop_search_by_age.show(truncate=False)

#### 5.2.6. Stop and Search by Gender

In [ ]:
stop_search_by_gender = police_stop_search_df \
    .groupBy("gender") \
    .agg(F.count("*").alias("total_searches")) \
    .orderBy(F.desc("total_searches"))

print("\nSummary by Gender:")
stop_search_by_gender.show(truncate=False)

#### 5.2.7. Stop and Search by Age Range and Gender

In [103]:
stop_search_by_age_gender = police_stop_search_df \
    .groupBy("age_range", "gender") \
    .agg(F.count("*").alias("total_searches")) \
    .orderBy(F.desc("total_searches"))

print("\nStop and Search by Age Range and Gender:")
stop_search_by_age_gender.show(20, truncate=False)


Stop and Search by Age Range and Gender:
+---------+------+--------------+
|age_range|gender|total_searches|
+---------+------+--------------+
|18-24    |Male  |89135         |
|over 34  |Male  |77731         |
|25-34    |Male  |71030         |
|10-17    |Male  |55622         |
|None     |Male  |42077         |
|over 34  |Female|12616         |
|25-34    |Female|7453          |
|18-24    |Female|6636          |
|18-24    |Other |4454          |
|None     |None  |4118          |
|18-24    |None  |4046          |
|10-17    |Female|3944          |
|None     |Female|3406          |
|25-34    |Other |3383          |
|over 34  |Other |3268          |
|25-34    |None  |3213          |
|over 34  |None  |3076          |
|None     |Other |2641          |
|10-17    |Other |2513          |
|10-17    |None  |2088          |
+---------+------+--------------+
only showing top 20 rows



Stop and Search by Age Range and Gender:
+---------+------+--------------+
|age_range|gender|total_searches|
+---------+------+--------------+
|18-24    |Male  |89135         |
|over 34  |Male  |77731         |
|25-34    |Male  |71030         |
|10-17    |Male  |55622         |
|None     |Male  |42077         |
|over 34  |Female|12616         |
|25-34    |Female|7453          |
|18-24    |Female|6636          |
|18-24    |Other |4454          |
|None     |None  |4118          |
|18-24    |None  |4046          |
|10-17    |Female|3944          |
|None     |Female|3406          |
|25-34    |Other |3383          |
|over 34  |Other |3268          |
|25-34    |None  |3213          |
|over 34  |None  |3076          |
|None     |Other |2641          |
|10-17    |Other |2513          |
|10-17    |None  |2088          |
+---------+------+--------------+
only showing top 20 rows


#### 5.2.8. Top Outcomes of Stop and Search

In [98]:
stop_search_by_outcome = police_stop_search_df \
    .groupBy("outcome") \
    .agg(F.count("*").alias("total_searches")) \
    .orderBy(F.desc("total_searches"))

# Calculate outcome percentages
total_searches = police_stop_search_df.count()
stop_search_outcome_pct = stop_search_by_outcome \
    .withColumn("percentage", F.round((col("total_searches") / total_searches) * 100, 2))

print("\nTop Outcomes with Percentages:")
stop_search_outcome_pct.show(10, truncate=False)



Top Outcomes with Percentages:
+-------------------------------+--------------+----------+
|outcome                        |total_searches|percentage|
+-------------------------------+--------------+----------+
|A no further action disposal   |277204        |68.81     |
|Arrest                         |66418         |16.49     |
|Community resolution           |39919         |9.91      |
|Penalty Notice for Disorder    |11446         |2.84      |
|Summons / charged by post      |5563          |1.38      |
|Caution (simple or conditional)|2291          |0.57      |
|None                           |7             |0.0       |
+-------------------------------+--------------+----------+



Income data vs Msao Map

In [100]:
# Top / Bottom income MSOAs + overall average (join income to MSOA map)
from pyspark.sql import functions as F

income_col = "total_annual_income_(£)"

income_geo_df = (
    total_income_df
    .join(
        map_msoa_df.select("msoa21cd", "msoa21nm", "lat", "long"),
        total_income_df["msoa_code"] == F.col("msoa21cd"),
        how="left",
    )
    .select(
        F.col("msoa_code").alias("msoa_code"),
        F.coalesce(F.col("msoa_name"), F.col("msoa21nm")).alias("msoa_name"),
        F.col(income_col).cast("double").alias("total_annual_income_gbp"),
        F.col("lat").cast("double").alias("lat"),
        F.col("long").cast("double").alias("long"),
    )
    .dropna(subset=["total_annual_income_gbp"])
)

income_geo_df.createOrReplaceTempView("income_geo")

# Top 10 highest-income locations
spark.sql("""
SELECT msoa_code, msoa_name, total_annual_income_gbp, lat, long
FROM income_geo
ORDER BY total_annual_income_gbp DESC
LIMIT 10
""").show(truncate=False)

# Bottom 10 lowest-income locations
spark.sql("""
SELECT msoa_code, msoa_name, total_annual_income_gbp, lat, long
FROM income_geo
ORDER BY total_annual_income_gbp ASC
LIMIT 10
""").show(truncate=False)

# Average income across all MSOAs
spark.sql("""
SELECT AVG(total_annual_income_gbp) AS avg_total_annual_income_gbp
FROM income_geo
""").show()

+---------+--------------------------+-----------------------+--------+--------+
|msoa_code|msoa_name                 |total_annual_income_gbp|lat     |long    |
+---------+--------------------------+-----------------------+--------+--------+
|E02000931|Wandsworth 009            |108100.0               |51.4606 |-0.15622|
|E02000001|City of London 001        |101800.0               |51.51562|-0.09349|
|E02000890|Tower Hamlets 027         |96600.0                |51.50671|-0.07076|
|E02000937|Wandsworth 015            |94000.0                |51.45231|-0.16795|
|E02000949|Wandsworth 027            |93800.0                |51.44086|-0.16982|
|E02006334|Elmbridge 018             |93500.0                |51.32983|-0.36945|
|E02000809|Southwark 003             |92800.0                |NULL    |NULL    |
|E02006325|Elmbridge 009             |92100.0                |51.3752 |-0.33779|
|E02000939|Wandsworth 017            |91600.0                |51.45308|-0.157  |
|E02000592|Kensington and Ch